In [42]:
import pandas as pd
import sys
import numpy as np

import operator
from models import Input, AspectOutput
from tqdm import tqdm
import re
from gensim.models import KeyedVectors
import json


def load_aspect_data_mebe(path):
    """

    :param path:
    :return:
    :rtype: (list of models.Input, list of models.AspectOutput)
    """
    inputs = []
    outputs = []
    df = pd.read_csv(path, encoding='utf-8')
    for _, r in df.iterrows():
        t = str(r['text']).strip()
        inputs.append(t)

        scores = r['label']
        sc = []
        for i in scores:
            if i == '1': sc.append(1)
            elif i == '0': sc.append(0)
        outputs.append(sc)
    outputs = np.array(outputs)
    return inputs, outputs
inputs_shopee, outputs_shopee = load_aspect_data_mebe('data/raw_data/mebe_shopee.csv')
inputs_tiki, outputs_tiki = load_aspect_data_mebe('data/raw_data/mebe_tiki.csv')
inputs = inputs_shopee+inputs_tiki


In [43]:
outputs_shopee[0]

array([1, 0, 0, 0, 0, 0, 0])

In [9]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(inputs_tiki, outputs_tiki, test_size=0.2, random_state=14)
# train = {'text': X_train, 'label': y_train.tolist()}
# test = {'text': X_test, 'label': y_test.tolist()}
# df_train = pd.DataFrame(train)
# df_test = pd.DataFrame(test)
# df_train.to_csv(r"C:\Users\acer\Documents\hoc tap\khóa luận\mebe_domain\data\train_mebe_tiki.csv")
# df_test.to_csv(r"C:\Users\acer\Documents\hoc tap\khóa luận\mebe_domain\data\test_mebe_tiki.csv")


In [33]:
import sklearn

In [44]:
with open('output_shopee.txt', 'w') as f:
    f.write(json.dumps(outputs_shopee.tolist()))
with open('output_tiki.txt', 'w') as f:
    f.write(json.dumps(outputs_tiki.tolist()))

In [45]:



def preprocess(inputs):
    """

    :param list of models.Input inputs:
    :return:
    :rtype: list of models.Input
    """
    corpus=[]
    for text in tqdm(inputs):
        text = text.lower()
        # text=re.sub(r'[^a-z0-9A-Z]'," ",text)
        text=re.sub(r'[0-9]{1}',"#",text)
        text=re.sub(r'[0-9]{2}','##',text)
        text=re.sub(r'[0-9]{3}','###',text)
        text=re.sub(r'[0-9]{4}','####',text)
        text=re.sub(r'[0-9]{5,}','#####',text)
        corpus.append(text)
    return corpus
pre_text = preprocess(inputs)
pre_text_shopee = preprocess(inputs_shopee)
pre_text_tiki = preprocess(inputs_tiki)
def get_vocab(corpus):
    vocab={}
    for text in tqdm(corpus):
        for word in text.split():
            try:
                vocab[word]+=1
            except KeyError:
                vocab[word]=1
    vocab=dict(sorted(vocab.items(),reverse=True ,key=lambda item: item[1]))
    return vocab

100%|███████████████████████████████████████████████████████████████████████████| 2940/2940 [00:00<00:00, 94992.44it/s]


In [38]:
model_embed = KeyedVectors.load_word2vec_format('../cc.vi.300.vec')


In [41]:
model_embed["giaohàng"]

array([-0.0034,  0.0118,  0.0262, -0.0376, -0.0155,  0.0124,  0.0152,
        0.002 ,  0.016 , -0.0012,  0.0043, -0.0206,  0.0036,  0.0083,
       -0.0458,  0.0203, -0.0114,  0.007 , -0.031 , -0.    , -0.0181,
       -0.0155, -0.0089,  0.0045,  0.0096,  0.0108,  0.001 , -0.017 ,
        0.0441, -0.0082, -0.0125, -0.0114, -0.0099, -0.0007,  0.0168,
        0.0161,  0.0199,  0.0121,  0.032 , -0.0159, -0.0072,  0.0349,
       -0.0175, -0.0378,  0.024 , -0.0055,  0.0367, -0.0074, -0.0574,
        0.014 ,  0.0364,  0.0064, -0.0141, -0.0245, -0.0045, -0.0409,
       -0.0693, -0.005 , -0.0612, -0.0144,  0.0142, -0.0003,  0.0182,
        0.0154,  0.016 , -0.0092,  0.0376, -0.0099,  0.0527,  0.0025,
        0.0091, -0.0118, -0.0134, -0.013 ,  0.006 ,  0.0077, -0.0016,
       -0.0075,  0.016 ,  0.0021, -0.06  , -0.002 , -0.0185,  0.0053,
        0.01  , -0.0136, -0.0547, -0.0223,  0.0136, -0.0069,  0.0051,
        0.023 , -0.0373, -0.0062,  0.007 ,  0.0433,  0.0356, -0.0177,
       -0.0094, -0.0

In [46]:

def replace_sym(word):
    original = ["đóng_gói", "dài_dài", "ưng_ý", "sản_phẩm_chất_lượng","kĩ_càng", "bao_bì", "nguyên_vẹn", 
                "bóp_méo", "nowship", "date_dài", "date_xa", "hạn_dùng", "túi_tiền", "băng_keo", "mã_vạch", "tin_dùng", "cực_kì"]
    sym = ["giao_hàng", "lâu_dài", "hài_lòng", "chất_lượng", "kĩ", "vỏ",
           "nguyên", "méo", "giao_hàng", "hạn", "hạn", "hạn", "giá_cả", "dính", "mã", "tin_tưởng", "rất"]
    for i in range(len(original)):
        if original[i] == word:
            return sym[i]
    return word
        

In [47]:
def check_voc(vocab,model):
    embed_words=[]
    out_vocab={}
    total_words=0
    total_text=0
    list_word_embed = list()

    for i in tqdm(vocab):
        temp = i
        try:
            i = replace_sym(i)
            print(type(i))
            i = i.replace("_", "")
            vec=model[i]
            embed_words.append(vec) # list các vector của những từ xuất hiện cả embed vocab và vocab dữ liệu
            total_words+=vocab[temp] # tổng số từ xuất hiện trong dữ liệu mà có trong cả embed vocab, tính cả từ giống nhau
            print("từ có trong embed:"+ i)
        except KeyError: ## những từ không xuất hiện trong embed model
            out_vocab[temp]=vocab[temp]
            total_text+=vocab[temp] # total text bằng chính tổng lượng từ trong dữ liệu
            print("từ không có trong embed" + temp)
    print("The {:.2f}% of vocabularies have Covered of corpus".format(100*len(embed_words)/len(vocab)))
    print("The {:.2f}% of total text had coverded ".format((100*total_words/(total_words+total_text))))
    return out_vocab

In [48]:
vocabulary=get_vocab(pre_text)
oov=check_voc(vocabulary,model_embed)
sort_oov=dict(sorted(oov.items(), key=operator.itemgetter(1),reverse=True))
print(dict(list(sort_oov.items())[:50]))

 12%|█████████▏                                                                   | 625/5236 [00:00<00:00, 6239.44it/s]

<class 'str'>
từ có trong embed:hàng
<class 'str'>
từ có trong embed:giao
<class 'str'>
từ có trong embed:nhanh
<class 'str'>
từ có trong embed:mua
<class 'str'>
từ có trong embed:tốt
<class 'str'>
từ có trong embed:giaohàng
<class 'str'>
từ có trong embed:sảnphẩm
<class 'str'>
từ có trong embed:bé
<class 'str'>
từ có trong embed:sữa
<class 'str'>
từ có trong embed:chấtlượng
<class 'str'>
từ có trong embed:cẩnthận
<class 'str'>
từ có trong embed:giá
<class 'str'>
từ có trong embed:ok
<class 'str'>
từ có trong embed:dùng
<class 'str'>
từ có trong embed:ủnghộ
<class 'str'>
từ có trong embed:#
<class 'str'>
từ có trong embed:nhận
<class 'str'>
từ có trong embed:bỉm
<class 'str'>
từ có trong embed:tã
<class 'str'>
từ có trong embed:date
<class 'str'>
từ có trong embed:mới
<class 'str'>
từ có trong embed:hộp
<class 'str'>
từ có trong embed:hàilòng
<class 'str'>
từ có trong embed:rẻ
<class 'str'>
từ có trong embed:sp
<class 'str'>
từ có trong embed:tặng
<class 'str'>
từ có trong embed:đúng
<

 42%|███████████████████████████████▌                                            | 2176/5236 [00:00<00:00, 6621.12it/s]

<class 'str'>
từ không có trong embedkĩ_lưỡng
<class 'str'>
từ không có trong embedchút_xíu
<class 'str'>
từ có trong embed:mạng
<class 'str'>
từ có trong embed:chitiết
<class 'str'>
từ có trong embed:hả
<class 'str'>
từ có trong embed:xước
<class 'str'>
từ có trong embed:meiji
<class 'str'>
từ có trong embed:qá
<class 'str'>
từ có trong embed:ơn
<class 'str'>
từ có trong embed:quantâm
<class 'str'>
từ có trong embed:you
<class 'str'>
từ không có trong embedđi_ngoài
<class 'str'>
từ không có trong embedgiải_đáp
<class 'str'>
từ có trong embed:ban
<class 'str'>
từ có trong embed:ỹ
<class 'str'>
từ có trong embed:cac
<class 'str'>
từ có trong embed:trảinghiệm
<class 'str'>
từ có trong embed:lừa
<class 'str'>
từ có trong embed:giùm
<class 'str'>
từ có trong embed:mê
<class 'str'>
từ có trong embed:điều
<class 'str'>
từ có trong embed:cham
<class 'str'>
từ có trong embed:nc
<class 'str'>
từ có trong embed:lýdo
<class 'str'>
từ có trong embed:nét
<class 'str'>
từ có trong embed:cuộc
<class 

 70%|█████████████████████████████████████████████████████▍                      | 3682/5236 [00:00<00:00, 6791.21it/s]

<class 'str'>
từ có trong embed:nhưý
<class 'str'>
từ có trong embed:binh
<class 'str'>
từ không có trong embedtốt_giống
<class 'str'>
từ có trong embed:is
<class 'str'>
từ có trong embed:star
<class 'str'>
từ có trong embed:rờ
<class 'str'>
từ có trong embed:om
<class 'str'>
từ có trong embed:dưỡng
<class 'str'>
từ có trong embed:heoheo
<class 'str'>
từ không có trong embedánnjdjr
<class 'str'>
từ không có trong embedsbhsjd
<class 'str'>
từ không có trong embedbạndn
<class 'str'>
từ không có trong embedbahhđm
<class 'str'>
từ không có trong embedbajsnmd
<class 'str'>
từ không có trong embednạnkdkd
<class 'str'>
từ không có trong embedbâjjdjmd
<class 'str'>
từ không có trong embedmajdjfmf
<class 'str'>
từ có trong embed:khoe
<class 'str'>
từ không có trong embedhạt_tiêu
<class 'str'>
từ không có trong embedgia_vị
<class 'str'>
từ có trong embed:rượu
<class 'str'>
từ có trong embed:muối
<class 'str'>
từ có trong embed:laz
<class 'str'>
từ không có trong embedạkkkkkkkkkkkkkkkkkkkkkkkkkkk

 83%|███████████████████████████████████████████████████████████████▎            | 4363/5236 [00:00<00:00, 6613.01it/s]

từ không có trong embeddkdjdjnenrjejrjjrjrjjrnr
<class 'str'>
từ có trong embed:biếnđộng
<class 'str'>
từ không có trong embedthêmmmmmm
<class 'str'>
từ có trong embed:hoản
<class 'str'>
từ không có trong embedsuuuuuuuuuuuuuuuuuuuuuuuuuuu
<class 'str'>
từ không có trong embedjdndbdbdndbdmdbsmsbsnsbsndjsnsnsbssnansnsns
<class 'str'>
từ có trong embed:nhed
<class 'str'>
từ có trong embed:chòng
<class 'str'>
từ không có trong embedkjxnxjxoxckncdjejisnx
<class 'str'>
từ không có trong embedccmcmlclvvn
<class 'str'>
từ không có trong embeddksvkfkđjehabvs
<class 'str'>
từ có trong embed:xc
<class 'str'>
từ không có trong embedidjdnksaoaijdbcnccj
<class 'str'>
từ có trong embed:đực
<class 'str'>
từ có trong embed:biét
<class 'str'>
từ không có trong embedchhats
<class 'str'>
từ không có trong embedngonnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
<class 'str'>
từ không có trong embedjdkfk
<class 'str'>
từ không có trong embedjsjdbbf
<class 'str'>
từ không có trong embedmacx
<class 'str'>
từ không có tron

100%|████████████████████████████████████████████████████████████████████████████| 5236/5236 [00:00<00:00, 7021.59it/s]


từ không có trong embedhahhshdhdjdjdshhdxhdebhsjdhhe
<class 'str'>
từ có trong embed:tổn
<class 'str'>
từ có trong embed:kinhnghiệm
<class 'str'>
từ có trong embed:niềng
<class 'str'>
từ không có trong embedyinhs
<class 'str'>
từ có trong embed:đángkể
<class 'str'>
từ không có trong embedchuyên_dụng
<class 'str'>
từ có trong embed:vậtliệu
<class 'str'>
từ không có trong embedgiúp_ích
<class 'str'>
từ có trong embed:lưugiữ
<class 'str'>
từ có trong embed:hạng
<class 'str'>
từ không có trong embedfdo
<class 'str'>
từ có trong embed:giới
<class 'str'>
từ có trong embed:chừng
<class 'str'>
từ có trong embed:sẳn
<class 'str'>
từ có trong embed:khuy
<class 'str'>
từ có trong embed:genki
<class 'str'>
từ có trong embed:concung
<class 'str'>
từ có trong embed:chủđộng
<class 'str'>
từ có trong embed:độngtác
<class 'str'>
từ có trong embed:pack
<class 'str'>
từ có trong embed:refund
<class 'str'>
từ có trong embed:đồnghồ
<class 'str'>
từ có trong embed:ngân
<class 'str'>
từ có trong embed:tồnkh

In [50]:
len(vocabulary)

5236

In [11]:
def get_word_index(vocab):
    word_index=dict((w,i+1) for i,w in enumerate(vocab.keys()))
    return word_index
def fit_one_hot(word_index,corpus):
    sent=[]
    for text in tqdm(corpus):
        li=[]
        for word in text.split():
            try:
                li.append(word_index[word])
            except KeyError:
                li.append(0)
        sent.append(li)
    return sent
# print(len(vocabulary))
word_index=get_word_index(vocabulary)
encode_data_shopee = fit_one_hot(word_index, pre_text_shopee)
encode_data_tiki = fit_one_hot(word_index, pre_text_tiki)

with open('encode_data_shopee.txt', 'w') as f:
    f.write(json.dumps(encode_data_shopee))
with open('encode_data_tiki.txt', 'w') as f:
    f.write(json.dumps(encode_data_tiki))
    

count=0
embedding_mat=np.zeros((len(vocabulary)+1,300)) # tạo ma trận weight của từ dựa trên từ điển embed
for word,i in tqdm(word_index.items()):
    word = word.replace("_", " ")
    try:
        vec=model_embed[word]
        embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue
# with open('modules/aspect/weights/embedding_mat.txt', 'w') as f:
#     f.write(json.dumps(embedding_mat.tolist()))
np.save('modules/aspect/weights/embedding_mat', embedding_mat)


print("Number of Out of Vocabulary",count)

100%|██████████████████████████████████████████████████████████████████████████| 4513/4513 [00:00<00:00, 229185.56it/s]

Number of Out of Vocabulary 1939
